In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Clone the repository
!git clone https://github.com/hyperopt/hyperopt-sklearn.git

# Change to the directory
%cd hyperopt-sklearn

# Install the package in editable mode
!pip install -e .


Cloning into 'hyperopt-sklearn'...
remote: Enumerating objects: 3023, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 3023 (delta 212), reused 276 (delta 196), pack-reused 2711 (from 1)
Receiving objects: 100% (3023/3023), 2.31 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (1914/1914), done.
/kaggle/working/hyperopt-sklearn
Obtaining file:///kaggle/working/hyperopt-sklearn
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 84.1 MB/s eta 0:00:00:00:01:01
  Building editable for hpsklearn (pyproject.toml) ... done
  Created wheel for hpsklearn: filename=hpsklearn-1.0.3-0.editable-py3-none-any.whl size=7085 sha256=cf3995e2595758ab1b4fef5ca54fedf3d59c8b0b1f68039c97a0c22fb5cb24f0
  Stored in directory: /tmp/p

In [4]:
!pip install openml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 5.0 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=a554ee13887cd3e45737ba54e40d97a06f31033ff502ba5a738f00a68d4c5976
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [8]:
import openml
import time
import numpy as np
import pandas as pd
from hpsklearn import HyperoptEstimator, any_regressor
from hyperopt import tpe
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# ✅ Define datasets to search (OpenML IDs)
dataset_ids = [ 8, 195,531,204]  # California Housing, Dataset 8, Dataset 531
results = []

# ✅ Iterate over datasets
for dataset_id in dataset_ids:
    print(f"\n🔍 Loading dataset {dataset_id} from OpenML...")
    dataset = openml.datasets.get_dataset(dataset_id)
    
    # Get data from OpenML dataset
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)
    
    # ✅ Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # ✅ Define Hyperopt-Sklearn Estimator
    hyperopt_automl = HyperoptEstimator(
        regressor=any_regressor('reg'),  # Search all regressors
        algo=tpe.suggest,                # Use TPE (Tree-structured Parzen Estimator)
        max_evals=50,                     # Increase trials for better performance
        trial_timeout=300                  # Limit time per model evaluation
    )
    
    # ✅ Train the model
    print(f"🚀 Training Hyperopt-Sklearn on dataset {dataset_id}...")
    start_time = time.time()
    hyperopt_automl.fit(X_train.to_numpy(), y_train.to_numpy())  # Convert to NumPy arrays
    fit_time = time.time() - start_time
    
    # ✅ Make predictions
    y_pred = hyperopt_automl.predict(X_test.to_numpy())
    
    # ✅ Evaluate performance
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # ✅ Store results
    result = {
        "Dataset ID": dataset_id,
        "Hyperopt R² Score": r2,
        "Hyperopt MSE": mse,
        "Hyperopt Time": fit_time,
        "Hyperopt Best Models": hyperopt_automl.best_model()
    }
    results.append(result)
    
    print(f"✅ Completed dataset {dataset_id} -> MSE: {mse:.4f}, R²: {r2:.4f}")

# ✅ Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("hyperopt_results.csv", index=False)

print("\n📁 Results saved to hyperopt_results.csv!")



🔍 Loading dataset 8 from OpenML...
🚀 Training Hyperopt-Sklearn on dataset 8...
 98%|█████████▊| 49/50 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:1275: RuntimeWarning: overflow encountered in square
  numerator = xp.sum(weight * (y_true - y_pred) ** 2, axis=0)



100%|██████████| 50/50 [00:01<00:00,  1.85s/trial, best loss: 0.6976104874059909]
✅ Completed dataset 8 -> MSE: 9.4551, R²: 0.1118

🔍 Loading dataset 195 from OpenML...
🚀 Training Hyperopt-Sklearn on dataset 195...
 98%|█████████▊| 44/45 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1608: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(



100%|██████████| 50/50 [00:02<00:00,  2.66s/trial, best loss: 0.1783250173225669]
✅ Completed dataset 195 -> MSE: 6224542.1875, R²: 0.6502

🔍 Loading dataset 531 from OpenML...
🚀 Training Hyperopt-Sklearn on dataset 531...
100%|██████████| 1/1 [05:00<00:00, 300.14s/trial, best loss=?]


AllTrialsFailed: 

In [6]:
import openml
import time
import numpy as np
import pandas as pd
from hpsklearn import HyperoptEstimator, any_regressor
from hyperopt import tpe
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# ✅ Define datasets to search (OpenML IDs)
dataset_ids = [ 204]  # California Housing, Dataset 8, Dataset 531
results = []

# ✅ Iterate over datasets
for dataset_id in dataset_ids:
    print(f"\n🔍 Loading dataset {dataset_id} from OpenML...")
    dataset = openml.datasets.get_dataset(dataset_id)
    
    # Get data from OpenML dataset
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)
    mask = X.isna().any(axis=1) | y.isna()

    # Drop those rows from both X and y
    X = X[~mask]
    y = y[~mask]
    
    # ✅ Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # ✅ Define Hyperopt-Sklearn Estimator
    hyperopt_automl = HyperoptEstimator(
        regressor=any_regressor('reg'),  # Search all regressors
        algo=tpe.suggest,                # Use TPE (Tree-structured Parzen Estimator)
        max_evals=50,                     # Increase trials for better performance
        trial_timeout=300                  # Limit time per model evaluation
    )
    
    # ✅ Train the model
    print(f"🚀 Training Hyperopt-Sklearn on dataset {dataset_id}...")
    start_time = time.time()
    hyperopt_automl.fit(X_train.to_numpy(), y_train.to_numpy())  # Convert to NumPy arrays
    fit_time = time.time() - start_time
    
    # ✅ Make predictions
    y_pred = hyperopt_automl.predict(X_test.to_numpy())
    
    # ✅ Evaluate performance
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # ✅ Store results
    result = {
        "Dataset ID": dataset_id,
        "Hyperopt R² Score": r2,
        "Hyperopt MSE": mse,
        "Hyperopt Time": fit_time,
        "Hyperopt Best Models": hyperopt_automl.best_model()
    }
    results.append(result)
    
    print(f"✅ Completed dataset {dataset_id} -> MSE: {mse:.4f}, R²: {r2:.4f}")

# ✅ Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv("hyperopt_results.csv", index=False)

print("\n📁 Results saved to hyperopt_results.csv!")



🔍 Loading dataset 204 from OpenML...
🚀 Training Hyperopt-Sklearn on dataset 204...
100%|██████████| 50/50 [00:00<00:00,  7.14trial/s, best loss: 0.8992372324175311]
✅ Completed dataset 204 -> MSE: 2674.6323, R²: -0.0648

📁 Results saved to hyperopt_results.csv!


In [7]:
result

{'Dataset ID': 204,
 'Hyperopt R² Score': -0.06479912778759478,
 'Hyperopt MSE': 2674.632252675257,
 'Hyperopt Time': 30.684821605682373,
 'Hyperopt Best Models': {'learner': ExtraTreeRegressor(criterion='friedman_mse', max_depth=3, max_features='log2',
                     max_leaf_nodes=15, random_state=3, splitter='best'),
  'preprocs': (Normalizer(),),
  'ex_preprocs': ()}}

In [1]:
import openml

ModuleNotFoundError: No module named 'openml'